# cuML Cheat Sheets sample code

(c) 2020 NVIDIA, Blazing SQL

Distributed under Apache License 2.0

## Imports

In [1]:
import cudf
import cuml
import numpy as np
import cupy as cp

## Sample dataset

In [2]:
df = cudf.DataFrame([
      (1.44, -0.17, -0.05,  1, 'a', 1,  0.02)
    , (1.93,  0.93,  0.59,  7, 'b', 1, -2.07)
    , (5.80, -1.02, -0.65, -4, 'a', 1,  2.08)
    , (1.94, -0.55, -0.25,  0, 'b', 1,  0.37)
    , (3.93,  2.59,  0.07,  2, 'c', 0, -0.01)
    , (1.63, -0.35, -0.13,  1, 'b', 1,  0.34)
    , (4.26, -0.09, -0.11,  1, 'b', 1, -0.16)
    , (2.29,  2.25, -0.15,  0, 'c', 0,  1.64)
    , (5.85, -1.64, -0.82, -5, 'a', 1,  2.11)
    , (1.49, -0.55, -0.16,  0, 'b', 1,  0.25)
], columns=['f_0', 'f_1', 'f_2', 'f_3', 'f_4', 'label', 'target'])
df

,f_0,f_1,f_2,f_3,f_4,label,target
0,1.44,-0.17,-0.05,1,a,1,0.02
1,1.93,0.93,0.59,7,b,1,-2.07
2,5.80,-1.02,-0.65,-4,a,1,2.08
3,1.94,-0.55,-0.25,0,b,1,0.37
4,3.93,2.59,0.07,2,c,0,-0.01
5,1.63,-0.35,-0.13,1,b,1,0.34
6,4.26,-0.09,-0.11,1,b,1,-0.16
7,2.29,2.25,-0.15,0,c,0,1.64
8,5.85,-1.64,-0.82,-5,a,1,2.11
9,1.49,-0.55,-0.16,0,b,1,0.25


---

# Data preprocessing

---

#### model_selection.train_test_split()

In [3]:
X_train, X_test, y_train, y_test = cuml.preprocessing.model_selection.train_test_split(
    df
    , 'label'
    , train_size=0.75
)

In [4]:
X_train, X_test, y_train, y_test = cuml.preprocessing.model_selection.train_test_split(
    df[['f_1', 'f_2']]
    , df['label']
    , train_size=0.75
    , stratify=True
)

#### preprocessing.LabelEncoder.LabelEncoder()

In [5]:
le = cuml.preprocessing.LabelEncoder()
le.fit_transform(df['f_4'])

0    0
1    1
2    0
3    1
4    2
5    1
6    1
7    2
8    0
9    1
dtype: uint8

In [6]:
encoder = le.fit(df['f_4'])
encoder.transform(df['f_4'])

0    0
1    1
2    0
3    1
4    2
5    1
6    1
7    2
8    0
9    1
dtype: uint8

In [7]:
le.inverse_transform(encoder.transform(df['f_4']))

0    a
1    b
2    a
3    b
4    c
5    b
6    b
7    c
8    a
9    b
dtype: object

#### preprocessing.LabelBinarizer()

In [8]:
lb = cuml.preprocessing.LabelBinarizer()
lb.fit_transform(df['label'])

,0,1
0,0,1
1,0,1
2,0,1
3,0,1
4,1,0
5,0,1
6,0,1
7,1,0
8,0,1
9,0,1


In [9]:
binarizer = lb.fit(df['label'])
binarizer.transform(df['label'])

,0,1
0,0,1
1,0,1
2,0,1
3,0,1
4,1,0
5,0,1
6,0,1
7,1,0
8,0,1
9,0,1


In [10]:
binarizer.inverse_transform(binarizer.transform(cp.asarray(df['label'])))

array([1, 1, 1, 1, 0, 1, 1, 0, 1, 1])

#### preprocessing.OneHotEncoder()

In [11]:
ohe = cuml.preprocessing.OneHotEncoder(sparse=False)

In [12]:
ohe = cuml.preprocessing.OneHotEncoder(sparse=False, drop='first')

In [13]:
ohe.fit_transform(df[['f_4']])

array([[0., 0.],
       [1., 0.],
       [0., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 0.],
       [1., 0.]])

In [14]:
encoder = ohe.fit(df[['f_4']])
encoder.transform(df[['f_4']])

array([[0., 0.],
       [1., 0.],
       [0., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 0.],
       [1., 0.]])

In [15]:
encoder.inverse_transform(encoder.transform(df[['f_4']]))

,f_4
0,a
1,b
2,a
3,b
4,c
5,b
6,b
7,c
8,a
9,b


#### preprocessing.TargetEncoder()

In [16]:
te = cuml.preprocessing.TargetEncoder()

In [17]:
te = cuml.preprocessing.TargetEncoder(
    n_folds=5
    , smooth=0.1
    , split_method='interleaved'
    , output_type='cupy'
)

In [18]:
te.fit_transform(df['f_4'], df['label'])

array([0.93793103, 0.95384615, 0.93793103, 0.96326531, 0.37894737,
       0.96326531, 0.95384615, 0.37894737, 0.93793103, 0.96326531])

In [19]:
encoder = te.fit(df['f_4'], df['label'])
te.transform(df['f_4'])

array([0.93793103, 0.95384615, 0.93793103, 0.96326531, 0.37894737,
       0.96326531, 0.95384615, 0.37894737, 0.93793103, 0.96326531])

#### feature_extraction.text.CountVectorizer()

In [20]:
string_ser = cudf.Series([
              'RAPIDS.ai is a suite of open-source libraries that allow you to run your end to end data science and analytics pipelines on GPUs.'
            , 'cuDF is a Python GPU DataFrame (built on the Apache Arrow columnar memory format)'
            , 'cuDF allows for loading, joining, aggregating, filtering, and otherwise manipulating tabular data using a DataFrame style API.'
            , '''If your workflow is fast enough on a single GPU or your data comfortably fits in memory on 
                 a single GPU, you would want to use cuDF.'''
            , '''If you want to distribute your workflow across multiple GPUs or have more data than you can fit 
                 in memory on a single GPU you would want to use Dask-cuDF'''
            , 'BlazingSQL provides a high-performance distributed SQL engine in Python'
            , 'BlazingSQL is built on the RAPIDS GPU data science ecosystem'
            , 'BlazingSQL lets you ETL raw data directly into GPU memory as a GPU DataFrame (GDF)'
            , 'Dask is a flexible library for parallel computing in Python'
])

In [21]:
from cuml.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [22]:
cv = CountVectorizer(
    stop_words='english'
    , ngram_range=(1,2)
)

In [23]:
cv.fit_transform(string_ser).toarray()

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [24]:
vectorizer = cv.fit(string_ser)
vectorizer.transform(string_ser).toarray()

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [25]:
vectorizer.vocabulary_

0                aggregating
1      aggregating filtering
2                         ai
3                   ai suite
4                      allow
               ...          
137          want distribute
138                 want use
139                 workflow
140            workflow fast
141        workflow multiple
Name: token, Length: 142, dtype: object

#### feature_extraction.text.HashingVectorizer()

In [26]:
hv = cuml.feature_extraction.text.HashingVectorizer()

In [27]:
hv = cuml.feature_extraction.text.HashingVectorizer(
    stop_words='english'
    , ngram_range=(1,2)
)

In [28]:
hv.fit_transform(string_ser).toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [29]:
vectorizer = hv.fit(string_ser)
vectorizer.transform(string_ser).toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

#### feature_extraction.text.TfidfVectorizer()

In [30]:
tv = cuml.feature_extraction.text.TfidfVectorizer()

In [31]:
tv = cuml.feature_extraction.text.TfidfVectorizer(
    stop_words='english'
    , ngram_range=(1,2)
)

In [32]:
tv.fit_transform(string_ser).toarray()

array([[0.        , 0.        , 0.18530017, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.20747681, 0.20747681, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [33]:
vectorizer = tv.fit(string_ser)
vectorizer.transform(string_ser).toarray()

array([[0.        , 0.        , 0.18530017, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.20747681, 0.20747681, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)